# Transformação de Dados

In [24]:
# Expandir a coluna num mesmo dataframe;

# Estratégia utilizada: Percorrer o DataFrame por linha, 
# normalizar a coluna aninhada e unir ao DataFrame anterior, 
# duplicando as linhas originais.

import pandas as pd
from pandas import json_normalize
from ast import literal_eval

# Leitura do arquivo JSON
df = pd.read_json('data.json')

# Declarar a coluna aninhada. 
# É possível fazer uma adaptação aqui para trabalharmos 
# com mais de uma coluna aninhada ao mesmo tempo

nested_column = 'ItemList' 

df_result = pd.DataFrame()

for index_row in df.index:

  # Selecionar um registro
  df_row = df[df.index == index_row]
  
  # Transformar a coluna aninhada da linha selecionada em DataFrame
  df_column_normalized = json_normalize(df_row[nested_column].explode())
  
  # Deletar a coluna aninhada do DataFrame original
  df_row = df_row.drop(nested_column, axis=1)

  # Unir o DataFrame original ao novo DataFrame da coluna aninhada
  df_joined = df_row.join(df_column_normalized, how='outer')
  
  # Preencher as demais linhas com os valores originais
  for column in df_row.columns:
    df_joined[column].fillna(df_row[column].values[0], inplace=True)

  # Unir o DataFrame obtido da linha com os demais resultados
  df_result = pd.concat([df_result, df_joined])
  
print(df_result)

               CreateDate         EmissionDate  Discount  NFeNumber  NFeID   ProductName  Value  Quantity
0  2021-05-24T20:21:34.79  2021-05-24T00:00:00       0.0      501.0    1.0          Rice  35.55         2
1  2021-05-24T20:21:34.79  2021-05-24T00:00:00       0.0      501.0    1.0         Flour  11.55         5
2  2021-05-24T20:21:34.79  2021-05-24T00:00:00       0.0      501.0    1.0          Bean  27.15         7
0  2021-05-24T20:21:34.79  2021-05-24T00:00:00       0.0      502.0    2.0        Tomate  12.25        10
1  2021-05-24T20:21:34.79  2021-05-24T00:00:00       0.0      502.0    2.0         Pasta   7.55         5
0  2021-05-24T20:21:34.79  2021-05-24T00:00:00       0.0      503.0    3.0          Beer   9.00         6
1  2021-05-24T20:21:34.79  2021-05-24T00:00:00       0.0      503.0    3.0  French fries  10.99         2
2  2021-05-24T20:21:34.79  2021-05-24T00:00:00       0.0      503.0    3.0     Ice cream  27.15         1


In [57]:
# Normalizar os itens dessa coluna de dicionário e dividí-los em dois dataframes separados, seguindo o modelo relacional.

# Observação: Pelo texto do cenário, comprendo que a coluna aninhada será dividia em dois DataFrames, totalizando três DataFrames. 
# Dessa forma, poderemos ter uma relação N:N e não somente 1:N, se tivéssemos apenas dois no total.
# Nesse entendimento, teremos três DataFrames, um para o cabeçalho da NFe, um para os itens da NFe, com apenas o código dos produtos, quantitativos e valor. 
# Optamos por manter o valor no detalhamento da nota, umas vez que podemos ter o mesmo produto com valores diferentes, dependendo dos reajustes de preço.

# Estratégia: 
# 1) A partir do DataFrame anterior, extrair o DataFrame do detalhamento da NFe
# 2) Percorrer o DataFrame do detalhamento da NFe e montar o DataFrame de Produtos

# Cabeçalho da NFe
df_cabecalho_NFe = df.drop('ItemList', axis=1)

# Detalhamento da NFe
df_detalhamento_NFe = df_result[['NFeID', 'ProductName', 'Value', 'Quantity']]

# Até este momento, temos dois DataFrames, temos o cabeçalho da NFe e seu detalhamento. Com cardinalidade 1:N.
# Entendo que este seria o formato ideal para ganho de produtividade nas análises, porém ainda não atende ao solicitado. 
# Uma vez que pode haver duplicidade no nome do produto, ou seja, não há ainda normalização por completo.

# Criação do DataFrame de Produtos, apenas com Código e Nome do produto
df_produtos = df_detalhamento_NFe['ProductName'].unique()
df_produtos = pd.DataFrame(df_produtos, columns = ['ProductName'])
df_produtos.insert(0, 'ProductID', range(0, len(df_produtos)))

# Normalização do DataFrame de detalhamento da NFe
df_detalhamento_NFe = df_detalhamento_NFe.merge(df_produtos,how='left', on='ProductName')
df_detalhamento_NFe = df_detalhamento_NFe.drop('ProductName', axis=1)

print('\nDataFrame de Produtos: \n')
print(df_produtos)
print('\nDataFrame do Cabeçalho da NFe: \n')
print(df_cabecalho_NFe)
print('\nDataFrame do Detalhamento da NFe: \n')
print(df_detalhamento_NFe)


DataFrame de Produtos: 

   ProductID   ProductName
0          0          Rice
1          1         Flour
2          2          Bean
3          3        Tomate
4          4         Pasta
5          5          Beer
6          6  French fries
7          7     Ice cream

DataFrame do Cabeçalho da NFe: 

               CreateDate         EmissionDate  Discount  NFeNumber  NFeID
0  2021-05-24T20:21:34.79  2021-05-24T00:00:00         0        501      1
1  2021-05-24T20:21:34.79  2021-05-24T00:00:00         0        502      2
2  2021-05-24T20:21:34.79  2021-05-24T00:00:00         0        503      3

DataFrame do Detalhamento da NFe: 

   NFeID  Value  Quantity  ProductID
0    1.0  35.55         2          0
1    1.0  11.55         5          1
2    1.0  27.15         7          2
3    2.0  12.25        10          3
4    2.0   7.55         5          4
5    3.0   9.00         6          5
6    3.0  10.99         2          6
7    3.0  27.15         1          7
